In [ ]:
%load_ext aiida
%aiida
import urllib.parse

import ipywidgets as ipw
import traitlets as trt
from IPython.display import clear_output

from surfaces_tools.helpers import HART_2_EV

In [ ]:
class FragmentSeparationViewer(ipw.VBox):
    pk = trt.Int(allow_none=True)

    def __init__(
        self,
    ):
        self.load_btn = ipw.Button(description="Load")
        self.load_btn.on_click(self.load)

        wrong_syntax = ipw.HTML(
            value="""<i class="fa fa-times" style="color:red;font-size:2em;" ></i> wrong syntax""",
            layout={"visibility": "hidden"},
        )

        self.pk_widget = ipw.Text(
            value="",
            description="Workflow PK:",
            style={"description_width": "initial"},
            layout={"width": "200px"},
        )

        def try_convert(value):
            wrong_syntax.layout.visibility = "hidden"
            try:
                return int(value)
            except ValueError:
                wrong_syntax.layout.visibility = "visible"
                return None

        trt.dlink((self.pk_widget, "value"), (self, "pk"), try_convert)
        self.output = ipw.Output()
        super().__init__(
            [ipw.HBox([self.pk_widget, self.load_btn, wrong_syntax]), self.output]
        )

    def load(self, _=None):
        with self.output:
            clear_output()
            if not self.pk:
                print("No correct pk provided")
                return
            self.node = load_node(self.pk)
            try:
                assert self.node.process_label == "Cp2kFragmentSeparationWorkChain"

            except Exception:
                print(
                    f"Expected Cp2kFragmentSeparationWorkChain, got {type(self.node)}"
                )
                return

            self.parse_workchain()
            print(self.print_results())

    def print_results(self):
        energies = self.node.outputs.energies
        output = ""
        output += f"""Adsorption energy (frozen): {round(energies["unrelaxed_separation_energy"],3)*HART_2_EV:0.3f} (eV)\n"""
        output += f"""Adsorption energy: {round(energies["separation_energy"],3)*HART_2_EV:0.3f} (eV)\n"""
        for fragment in self.fragment_labels:
            energy = (
                energies[fragment]["unrelaxed"] - energies[fragment]["relaxed"]
            ) * HART_2_EV
            output += f"""Deformation energy for the "{fragment}" fragment: {round(energy, 3)} (eV)\n"""
        return output

    def parse_workchain(self):
        self.fragment_labels = sorted(self.node.inputs.fragments.keys())
        self.fragment_labels.remove("all")

In [ ]:
viewer = FragmentSeparationViewer()
display(viewer)

In [ ]:
# Load the URL after everything is set up.
try:
    url = urllib.parse.urlsplit(jupyter_notebook_url)
    viewer.pk_widget.value = urllib.parse.parse_qs(url.query)["pk"][0]
    viewer.load()
except Exception as exception:
    print(exception)